# Using Amazon Web Services (AWS)

## Objectives

- Describe core AWS services & concepts
- Configure your laptop to use AWS
- Use SSH key to access EC2 instances
- Launch & access EC2
- Access S3

## What are the benefits of AWS and other cloud services?

<img src='assets/cloud_benefits.png'>

## Amazon vs. Other Cloud Services

<img src='assets/public_cloud_adoption.png'>

## AWS Storage + Execution

What are the primary services that Amazon AWS offers?

Name | Full Name | Service
---|---|---
S3 | Simple Storage Service | Storage
EC2 | Elastic Compute Cloud | Execution
EBS | Elastic Block Store | Storage attached to EC2 instances

### Pop Quiz

<details>
<summary>Q: I want to store some video files on the web. Which Amazon service should I use?</summary>
A: S3
</details>

<details>
<summary>Q: I just created an iPhone app which needs to store user profiles on the web somewhere. Which Amazon service should I use?</summary>
A: S3
</details>

<details>
<summary>Q: I want to create a web application that uses Javascript in the backend along with a MongoDB database. Which Amazon service should I use?</summary>
A: S3 + EC2 + EBS
</details>

## S3 vs. EBS

What is the difference between S3 and EBS? Why would I use one versus the other?

Feature | S3 | EBS
---|---|---
Can be accessed from | Anywhere on the web;<br/>any EC2 instance | Specific availability zone;<br/>EC2 instance attached to it
Pricing | Less expensive;<br/>Storage (3¢/GB);<br/>Use (1¢/10,000 requests) | More expensive;<br/>Storage (3¢/GB) [+ IOPS]
Latency | Higher | Lower
Throughput | Usually more | Usually less
Performance | Slightly worse |Slightly better
Max volume size | Unlimited | 16 TB
Max file size | 5 TB | 16 TB

### Pop Quiz

<details>
<summary>Q: What is latency?</summary>
A: Latency is the time it takes between making a request and the start of a response.
</details>

<details>
<summary>Q: Which is better?  Higher latency or lower?</summary>
A: Lower is better.
</details>

<details>
<summary>Q: Why is S3 latency higher than EBS?</summary>
A: One reason is that EBS is in the same availability zone.
</details>

## Amazon S3

### Buckets and Files

What is a bucket?
- A bucket is a container for files.
- Think of a bucket as a logical grouping of files like a sub-domain.
- A bucket can contain an arbitrary number of files.

How large can a file in a bucket be?
- A file in a bucket can be 5 TB.

### Bucket Names

What are best practices on naming buckets?
- Bucket names should be DNS-compliant.
- They must be at least 3 and no more than 63 characters long.
- They must be a series of one or more labels, separated by a single period. 
- Bucket names can contain lowercase letters, numbers, and hyphens. 
- Each label must start and end with a lowercase letter or a number.
- Bucket names must not be formatted as an IP address (e.g., 192.168.5.4).

What are some examples of valid bucket names?
- `myawsbucket`
- `my.aws.bucket`
- `myawsbucket.1`

What are some examples of invalid bucket names? 
- `.myawsbucket`
- `myawsbucket.`
- `my..examplebucket`

#### Pop Quiz

<details>
<summary>Q: Why are these bucket names invalid?</summary>
A: Bucket names cannot start or end with a period. And they cannot have a multiple periods next to each other.
</details>

### Creating Buckets

Q: How can I create a bucket?

- Create an access key and secret key here: https://console.aws.amazon.com/iam/home?region=us-east-1#/security_credential , make sure to save the `rootkey.csv` file.
- Create a file called `~/.aws/credentials` (on Linux/Mac) or `%USERPROFILE%\.aws\credentials` (on Windows), and insert the following code into it. Replace `ACCESS_KEY` and `SECRET_KEY` with the keys you got from Amazon.
  
```
[default]
aws_access_key_id = ACCESS_KEY
aws_secret_access_key = SECRET_KEY
```

- Create a connection to S3.

In [1]:
import boto

conn = boto.connect_s3()

conn

S3Connection:s3.amazonaws.com

- List all the buckets.

In [2]:
conn.get_all_buckets()

[<Bucket: moses-new-bucket>,
 <Bucket: moses-whatever>,
 <Bucket: moses.testing.bucket>]

- Create new bucket.

In [3]:
import os

username = os.environ['USER']
print 'username =', username

bucket_name = username + "-new-bucket"
print 'bucket_name =', bucket_name

bucket = conn.create_bucket(bucket_name)
print 'bucket =', bucket

username = moses
bucket_name = moses-new-bucket
bucket = <Bucket: moses-new-bucket>


### Upgrading Boto

Q: Boto is not able to find the credentials. How can I fix this?
- Older versions of Boto were not able to read the credentials file.

### Adding Files

Q: How can I add a file to a bucket?

- List files.

In [4]:
bucket.get_all_keys()

[<Key: moses-new-bucket,hello-world.txt>]

- Add file.

In [5]:
text_file = bucket.new_key('hello-world.txt')
print text_file

text_file.set_contents_from_string('Hello World!')

<Key: moses-new-bucket,hello-world.txt>


12

- List files again; new file should appear.

In [6]:
bucket.get_all_keys()

[<Key: moses-new-bucket,hello-world.txt>]

Q: How can I get a file from a bucket?

In [7]:
another_file = bucket.get_key('hello-world.txt')
print another_file

another_file.get_contents_as_string()

<Key: moses-new-bucket,hello-world.txt>


'Hello World!'

### Creating Buckets With Periods

Q: How can I create a bucket in Boto with a period in the name?
- There is a bug in Boto that causes `create_bucket` to fail if the bucket name has a period in it. 
- Try creating the bucket with a period in its name. This should fail.

In [8]:
bucket_name_with_period = bucket_name + ".1.2.3"

bucket_with_period = conn.create_bucket(bucket_name_with_period)

bucket_with_period

CertificateError: hostname 'moses-new-bucket.1.2.3.s3.amazonaws.com' doesn't match either of '*.s3.amazonaws.com', 's3.amazonaws.com'

- To get around this run this code snippet.

In [9]:
import ssl

if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

- Now try creating the bucket with a period in its name and it should work.

In [10]:
bucket_name_with_period = bucket_name + ".1.2.3"

bucket_with_period = conn.create_bucket(bucket_name_with_period)

bucket_with_period

<Bucket: moses-new-bucket.1.2.3>

- Now lets delete the bucket.

In [11]:
bucket_with_period.delete()

- For more details see <https://github.com/boto/boto/issues/2836>.

### Access Control

Q: I want to access my S3 file from a web browser without giving my access and secret keys. How can I open up access to the file to anyone?
- You can set up Access Control Lists (ACLs) at the level of the bucket or at the level of the individual objects in the bucket (folders, files).

Q: What are the different ACL policies?

ACL Policy | Meaning
---|---
`private` | No one else besides owner has any access rights.
`public-read` | Everyone has read access.
`public-read-write` | Everyone has read/write access.
`authenticated-read` | Registered Amazon S3 users have read access.

Q: What does `read` and `write` mean for buckets and files?
- Read access to a file lets you read the file.
- Read access to a bucket or folder lets you see the names of the files inside it.

#### Pop Quiz

<details>
<summary>Q: If a bucket is `private` and a file inside it is `public-read` can I view it through a web browser?</summary>
A: Yes. Access to the file is only determined by its ACL policy.
</details>

<details>
<summary>Q: If a bucket is `public-read` and a file inside it is `private` can I view the file through a web browser?</summary>
A: No, you cannot. However, if you access the URL for the bucket you will see the file listed.
</details>

### Applying Access Control

Q: How can I make a file available on the web so anyone can read it?
- Create a file with a specific ACL.

In [12]:
secret_file = bucket.new_key('yet-another-hello-world.txt')

secret_file.set_contents_from_string('Hello World!', policy = 'private')

12

- Try reading the file.

In [13]:
file_url = 'http://s3.amazonaws.com/' + bucket_name + '/yet-another-hello-world.txt'

print file_url

!curl $file_url

http://s3.amazonaws.com/moses-new-bucket/yet-another-hello-world.txt
<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>AccessDenied</Code><Message>Access Denied</Message><RequestId>A0D6CF0FAF10F1AA</RequestId><HostId>StgOBQDQUx6YkU44deHRKC5/vKL2xcwo3bWELGCvplqab5prklN3aCEUscvle0PAkzBanGhSaQU=</HostId></Error>

- Now change its ACL.

In [14]:
secret_file.set_acl('public-read')

!curl $file_url

Hello World!

- Also you can try accessing the file through the browser.
- If you do not specify the ACL for a file when you set its contents, the file is `private` by default.

### S3 Files to URLs

Q: How can I figure out the URL of my S3 file?
- As above, you can compose the URL using the region, bucket, and filename. 
- For 'N. Virginia' the general template for the URL is `http://s3.amazonaws.com/BUCKET/FILE`.
    - Region-specific endpoint is `http://s3-AWSREGION.amazonaws.com/BUCKET/FILE`
- You can also find the URL by looking at the file on the AWS web console.

### Deleting Buckets

Q: How can I delete a bucket?
- Try deleting a bucket containing files. What happens?

In [15]:
print conn.get_all_buckets()

[<Bucket: moses-new-bucket>, <Bucket: moses-whatever>, <Bucket: moses.testing.bucket>]


In [16]:
bucket

<Bucket: moses-new-bucket>

In [17]:
bucket.delete()

S3ResponseError: S3ResponseError: 409 Conflict
<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>BucketNotEmpty</Code><Message>The bucket you tried to delete is not empty</Message><BucketName>moses-new-bucket</BucketName><RequestId>CD6D596DD328A95D</RequestId><HostId>c++RblSZ5kL801U6wJQuMrYXzt+IlofQDCVh83VxiKu1T0zzLsyGy3cxYj35n7/trenYMxmhHOQ=</HostId></Error>

- To delete the bucket first delete all the files in it.

In [18]:
for key in bucket.get_all_keys(): 
    key.delete()

- Then delete the bucket.

In [19]:
print 'Before bucket deletion'
print conn.get_all_buckets()

bucket.delete()

print 'After bucket deletion'
print conn.get_all_buckets()

Before bucket deletion
[<Bucket: moses-new-bucket>, <Bucket: moses-whatever>, <Bucket: moses.testing.bucket>]
After bucket deletion
[<Bucket: moses-whatever>, <Bucket: moses.testing.bucket>]


## Amazon EC2

### Regions

Q: What are *AWS Regions*?
- AWS is hosted in different geographic locations worldwide. 
- For example, there are 4 regions in the US.

Region | Name | Location 
---|---|--- 
us-east-1 | US East | N. Virginia
us-east-2 | US East 2 | Ohio
us-west-1 | US West | N. California
us-west-2 | US West 2 | Oregon

Q: How should I choose a region?
- N. Virginia or `us-east-1` is the default region for EC2.
- Using a region other than N. Virginia requires additional configuration.
- If you are not sure, choose N. Virginia.

### Availability Zones

Q: What are *AWS Availability Zones*?

- Regions are divided into isolated availability zones for fault tolerance.
- Availability zones run on physically separate hardware and infrastructure.
- They do not share hardware, generators, or cooling equipment. 
- Availability zones are assigned automatically to your EC2 instances based on your user ID.

<img src='assets/aws_regions.png'>

<details>
<summary>Q: Is it possible for two separate users to coordinate and land on the same availability zone?</summary>
1. Availability zones are assigned automatically by the system.
2. It is not possible for two AWS users to coordinate and be hosted on the same availability zone.
</details>

### Connecting to EC2

Q: How can I connect to an EC2 instance?
- Login to the AWS console.
- Navigate: EC2 > Launch Instance > Community AMIs > Search community AMIs > `ami-a4c7edb2` (An Amazon AMI)
- View the instance and get its Public DNS.
    - This should look something like `ec2-34-229-96-155.compute-1.amazonaws.com`.
- Use this command to connect to it.
    - `ssh -X -i ~/.ssh/keypair.pem user@domain`
    - Here is an example:
        - `ssh -X -i ~/.ssh/keypair.pem ec2-user@ec2-34-229-96-155.compute-1.amazonaws.com`
- Make sure you replace the Public DNS value below with the value you have for your instance.

### Copying Files to EC2

Q: How can I copy files to the EC2 instance?
- To copy a file `myfile.txt` to EC2, use a command like this.
    - `scp -i ~/.ssh/keypair.pem myfile.txt user@domain:`
- To copy a directory `mydir` recursively to EC2, use a command like this. 
    - `scp -i ~/.ssh/keypair.pem -r mydir user@domain:`

#### Pop Quiz

<details>
<summary>Q: When you copy a file to EC2 with `scp` will this show up in S3?</summary>
A: No. The file will be stored on the disk on the EC2 instance. It will not be in S3.
</details>